In [0]:
# PIE CHART POR VALOR APROVADO — FUNCIONA COM QUALQUER NÚMERO DE ESTADOS!
import pandas as pd
import plotly.express as px

# LEITURA
df = spark.sql("select * from classes.gold.df_aprovacao_internacao_por_municipio").toPandas()

# TRATAMENTO ROBUSTO
df = df.dropna(subset=['municipio_id', 'municipio'])
df['municipio_id'] = pd.to_numeric(df['municipio_id'], errors='coerce')
df = df.dropna(subset=['municipio_id'])
df['municipio_id'] = df['municipio_id'].astype(int)

df['quantidade_aprovada'] = pd.to_numeric(df['quantidade_aprovada'], errors='coerce').fillna(0)
df['valor_aprovado'] = pd.to_numeric(df['valor_aprovado'], errors='coerce').fillna(0)

# MAPEAMENTO COMPLETO DOS ESTADOS BRASILEIROS
estados_br = {
    '11': 'RO', '12': 'AC', '13': 'AM', '14': 'RR', '15': 'PA', '16': 'AP', '17': 'TO',
    '21': 'MA', '22': 'PI', '23': 'CE', '24': 'RN', '25': 'PB', '26': 'PE', '27': 'AL',
    '28': 'SE', '29': 'BA', '31': 'MG', '32': 'ES', '33': 'RJ', '35': 'SP',
    '41': 'PR', '42': 'SC', '43': 'RS', '50': 'MS', '51': 'MT', '52': 'GO', '53': 'DF'
}

df['uf'] = df['municipio_id'].astype(str).str[:2].map(estados_br)

# AGREGA POR ESTADO
por_uf = (
    df.groupby('uf')
    .agg({
        'quantidade_aprovada': 'sum',
        'valor_aprovado': 'sum'
    })
    .reset_index()
    .sort_values('valor_aprovado', ascending=False)
)

# ADICIONA "Outros" se houver estados com menos de 1%
total_valor = por_uf['valor_aprovado'].sum()
por_uf['percentual'] = por_uf['valor_aprovado'] / total_valor * 100

principais = por_uf[por_uf['percentual'] >= 1.0]
outros = por_uf[por_uf['percentual'] < 1.0]

if len(outros) > 0:
    outros_row = pd.DataFrame([{
        'uf': 'Outros',
        'valor_aprovado': outros['valor_aprovado'].sum(),
        'quantidade_aprovada': outros['quantidade_aprovada'].sum(),
        'percentual': outros['percentual'].sum()
    }])
    por_uf_final = pd.concat([principais, outros_row], ignore_index=True)
else:
    por_uf_final = principais

# PIE CHART FINAL — MOSTRA TODOS OS ESTADOS (ou "Outros")
fig = px.pie(
    por_uf_final,
    names='uf',
    values='valor_aprovado',
    title="<b>Distribuição Nacional do Valor Aprovado por Estado (SUS)</b><br>"
          "<sup>Total: R$ {:,.2f} bilhões | {} estados representados</sup>".format(
              total_valor / 1e9, len(principais) if 'Outros' in por_uf_final['uf'].values else len(por_uf_final)
          ),
    hole=0.5,
    color_discrete_sequence=px.colors.qualitative.Bold + px.colors.sequential.Plasma,
    height=750
)

fig.update_traces(
    textposition="inside",
    textinfo="percent+label",
    hovertemplate="<b>%{label}</b><br>Valor: R$ %{value:,.2f}<br>Quantidade: %{customdata[0]:,.0f}<extra></extra>",
    marker=dict(line=dict(color="white", width=3)),
    customdata=por_uf_final[['quantidade_aprovada']]
)

fig.update_layout(
    title_x=0.5,
    font=dict(size=14),
    legend_title="Estados",
    showlegend=True,
    margin=dict(t=130, b=80)
)

fig.show()